# Study of entangled tori

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

from sklearn import datasets
import plotly.express as px

from gdeep.create_nets import Net, SaveLayerOutput
from gdeep.create_nets import train_classification_nn, Layers_list, get_activations
from gdeep.plotting import  persistence_diagrams_of_activations, plot_persistence_diagrams, betti_plot_layers
from gdeep.plotting import plot_PCA_activations
from gdeep.create_data import make_torus_point_cloud2, Rotation

from fastai.tabular.all import *

import pathlib



from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces

## Generate 9x2 entangled tori dataset

In [ ]:
data1 , labels1 = make_torus_point_cloud2(0, 100, 0.1, Rotation(0,1,0), [0,0,0], 2.,.5)
data2 , labels2 = make_torus_point_cloud2(1, 100, 0.1, Rotation(1,2,180), [2,0,0], 2.,.5)


translations = [[i*10,j*10,k*10] for i in range(3) for j in range(3) for k in range(3)]

tori_entangled = np.append(data1,data2,axis=0)
labels = np.append(labels1,labels2)

data = tori_entangled
l = labels
        
for t in translations:
    if t != [0,0,0]:
        data = np.append(data, np.add(tori_entangled,t),axis=0)
        l = np.append(l, labels)
    
    
df_fix=pd.DataFrame(data[0::50,:],columns=["x","y","z"])
df_fix["label"]=l[0::50]
px.scatter_3d(df_fix,x="x",y="y",z="z",color="label")

## Initialize and train model or directly import it

In [ ]:
model = Net(0, arch = [3,15,15,15,15,15,15,15,15,15,15]) 
model.load_state_dict(torch.load(pathlib.Path('model_tori').absolute()))
model

## Initialize hooks to save layer activation

In [ ]:
# consider the neural network circle_detect_nn
circle_detect_nn = model

# List layers of circle_detect_nn
print(f'Layers of circle_detect_nn: {list(circle_detect_nn.modules())[1:]}')

layer_1 = list(circle_detect_nn.modules())[1]


# creat a layer for the first layer
# for that we first have to creat an instance of the class
# SaveLayerOutput we want to save the activations into

activations_layer_1_saver = SaveLayerOutput()

# add the __call__ function of activations_layer_1_saver
# as a hook to layer_1
# note activations_layer_1_saver() is the same as
# activations_layer_1_saver.__call__()

handle_layer_1 = layer_1.register_forward_hook(activations_layer_1_saver)

# switch to eval mode
circle_detect_nn.eval()

# make a forward pass
circle_detect_nn.forward(x_cat=None, x_cont=torch.tensor([[1,1,1]]).float())

# get activation of layer 1
print(f'activation layer 1: {activations_layer_1_saver.get_outputs()[-1]}')
print(f'activation layer 1 shape: {activations_layer_1_saver.get_outputs()[-1].shape}')

# remove handle of layer 1
handle_layer_1.remove()

## Compute and visualize the PCA, PD and Betti Surfaces of the image of the data in each layers

In [ ]:
# Plot PCA of layerwise activations for the training set


plot_PCA_activations(model, data[0::10,:], l[0::10], n_components=3)


In [ ]:

# Persistence diagrams of layerwise activations for 1/50 of the training set

persistence_diagrams = persistence_diagrams_of_activations(model, tori_entangled[0::50,:], homology_dimensions=[0,1,2], max_edge_length = 10, k = -1)



In [ ]:
plot_persistence_diagrams(persistence_diagrams)

In [ ]:
betti_plot_layers(persistence_diagrams)

# Train your own model if needed

In [ ]:
learn = train_classification_nn(nn=Net(0,[3,15,15,15,15,15,15,15,15,15,15]), X=data, y=l, lr=0.001, n_epochs=3, bs=32)

In [ ]:

model = learn.model


## Save your own model after training

In [ ]:
torch.save(learn.model.state_dict(), pathlib.Path('model_tori_new').absolute())